In [91]:
import os
import yaml

from google.oauth2 import service_account
import os
import yaml
import git

## source vars

path = os.path.expanduser('~')

profile_pass = os.path.join(path,".droughty/profile.yaml")

with open(profile_pass) as f:
    lookml_config = yaml.load(f, Loader=yaml.FullLoader)

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return (git_root)
    
    
git_def_path = get_git_root(os.getcwd())

git_path = git_def_path

filename = 'droughty_project.yaml'

droughty_project = os.path.join(git_path,filename)

with open(droughty_project) as f:
    enviroment_project = yaml.load(f, Loader=yaml.FullLoader)



for key,value in enviroment_project.items():
    
    if key == 'profile':

        if lookml_config[value]['warehouse_name'] == 'big_query':

            if value in lookml_config:

        ## global vars 

                warehouse_name =  lookml_config[value]['warehouse_name']
                project_name =  lookml_config[value]['project_name']
                schema_name =  lookml_config[value]['schema_name']
                test_schemas = lookml_config[value]['test_schemas']
                
                warehouse_schema =   """

                with source as (

                    select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`

                    )

                    select * from source

                """.format(project_name,schema_name)

                dbml_reference_dict = """


                with source as (

                select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`

                ),

                    pks as (
                    select 
                    table_name as pk_table_name,
                    column_name as pk_column_name,
                    trim(column_name, "_pk") as pk_sk,
                    from source
                    where column_name like '%pk%'
                    ),

                    fks as (
                    select
                    table_name as fk_table_name,
                    column_name as fk_column_name,
                    trim(column_name, "_fk") as fk_sk,
                    from source
                    where column_name like '%fk%'

                    ),

                    references as (
                    select * from pks

                    inner join fks on pks.pk_sk = fks.fk_sk

                    )

                    select 
                    
                    *except (pk_column_name,pk_table_name),

                    case when pk_column_name is null
                        then 'not_available'
                    else pk_column_name
                    end as pk_column_name,

                    case when pk_table_name is null
                        then 'not_available'
                    else pk_table_name
                    end as pk_table_name                   
                    from source

                    left join references on source.column_name = references.fk_column_name and references.fk_table_name = source.table_name
                
                """.format(project_name,schema_name)

        elif lookml_config[value]['warehouse_name'] == 'snowflake':
            
            if value in lookml_config:

                warehouse_name =  lookml_config[value]['warehouse_name']
                project_name =  lookml_config[value]['project_name']
                schema_name =  lookml_config[value]['schema_name']
                test_schemas = lookml_config[value]['test_schemas']
                database = lookml_config[value]['database']

                snowflake_schema = '''

                select * from {0}.information_schema.columns;

                '''.format(database)

    ## warehouse test schemas

test_warehouse_schema =   """

        with source_1 as (

            select * from `{0}.{1}.INFORMATION_SCHEMA.COLUMNS`

            ),

        source_2 as (

        select * from `{0}.{2}.INFORMATION_SCHEMA.COLUMNS`
        
        ),
        
        source_3 as (

        select * from `{0}.{3}.INFORMATION_SCHEMA.COLUMNS`
        
        ),
        
        unioned as (

        select * from source_1
        
        union all
        
        select * from source_2
        
        union all
        
        select * from source_3
        
        )

        select * from unioned

""".format(project_name,test_schemas[0],test_schemas[1],test_schemas[2])

explores = (enviroment_project.get("explores"))

## 

explore_tables = []

for key,value in explores.items():

    for key,value in value.items():

        explore_tables.append(value)

## reduce explore_tables from array to single list
        
single_list_tables = [i[0] for i in explore_tables]


flat_list = []
for sublist in explore_tables:
    for item in sublist:
        flat_list.append(item)
        
final_list = []
for x in flat_list:
    final_list.append("'" + x + "'")
    
join_key_list = ['merge_counts_fk','merge_counts_pk']

params = {
    'project_id': project_name,
    'schema_id': schema_name, 
    'table_names': final_list,
    'table_names_unqouted': flat_list,
    'pk_fk_join_key_list': join_key_list
    
}


user_transaction_template = '''

with source as (

select * from `{{project_id}}.{{schema_id}}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
where table_name in 
{% for tables in table_names [2:] %} {{table_names | inclause }} {% endfor %}

),

{% for value in table_names_unqouted  %}

explore_table_row_count_{{ value | sqlsafe }} as (

select 

'{{ value | sqlsafe }}' as table_name,
count(*) as {{ value | sqlsafe }}_row_count

from `{{project_id}}.{{schema_id}}.{{ value | sqlsafe }}`

group by 1

),

{% endfor %}


merge_counts as (

select

source.table_name,

{% for value in table_names_unqouted  %}

{{ value | sqlsafe }}_row_count

{% if not loop.last %},{% endif %}

{% endfor %}

from source 

{% for value in table_names_unqouted  %}

left join 

explore_table_row_count_{{value | sqlsafe }} on source.table_name = explore_table_row_count_{{value | sqlsafe }}.table_name

{% endfor %}

group by 1,2,3,4

),

pks as (
    select 
    table_name as pk_table_name,
    column_name as pk_column_name,
    trim(column_name, "_pk") as pk_sk,
    from source
    where column_name like '%%pk%%'
),

fks as (
    select
    table_name as fk_table_name,
    column_name as fk_column_name,
    trim(column_name, "_fk") as fk_sk,
    from source
    where column_name like '%%fk%%'

)


select 

pk_table_name,
pk_column_name,
fk_table_name,
fk_column_name,

{% for value in table_names_unqouted  %}

        {% for join_keys in pk_fk_join_key_list [2:] %}

            {% set item_1 = table_names_unqouted[loop.index-1] %}
            {% set item_2 = pk_fk_join_key_list[loop.index-1] %}

         {{item_1}}.{{item_2}}_row_count

        {% if not loop.last %},{% endif %}

    {% endfor %}

{% endfor %}

from pks

inner join fks on pks.pk_sk = fks.fk_sk

{% for merge_values in pk_fk_join_key_list  %}

inner join merge_counts on {{merge_values | sqlsafe }}.table_name = pks.pk_table_name

{% endfor %}


'''

In [92]:
explores

{'sales_explore': {'parent_table': ['wh_marketing_website_event_tracks_fact'],
  'dimensions': ['wh_marketing_website_users_dim'],
  'facts': ['wh_marketing_website_event_pages_fact']}}

In [12]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')
query, bind_params = j.prepare_query(user_transaction_template,params)

In [13]:
print(query % bind_params)



with source as (

select * from `ra-development.jordan_analytics_dev.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
where table_name in 
 ('wh_marketing_website_event_pages_fact','wh_marketing_website_users_dim','wh_marketing_website_event_tracks_fact') 

),



explore_table_row_count_wh_marketing_website_event_pages_fact as (

select 

'wh_marketing_website_event_pages_fact' as table_name,
count(*) as wh_marketing_website_event_pages_fact_row_count

from `ra-development.jordan_analytics_dev.wh_marketing_website_event_pages_fact`

group by 1

),



explore_table_row_count_wh_marketing_website_users_dim as (

select 

'wh_marketing_website_users_dim' as table_name,
count(*) as wh_marketing_website_users_dim_row_count

from `ra-development.jordan_analytics_dev.wh_marketing_website_users_dim`

group by 1

),



explore_table_row_count_wh_marketing_website_event_tracks_fact as (

select 

'wh_marketing_website_event_tracks_fact' as table_name,
count(*) as wh_marketing_website_event_tracks_fact_r

In [15]:
enviroment_project

{'profile': 'ra-development',
 'explores': {'sales_explore': {'parent_table': ['wh_marketing_website_event_pages_fact'],
   'dimensions': ['wh_marketing_website_users_dim'],
   'facts': ['wh_marketing_website_event_tracks_fact']}}}

In [16]:
explores

{'sales_explore': {'parent_table': ['wh_marketing_website_event_pages_fact'],
  'dimensions': ['wh_marketing_website_users_dim'],
  'facts': ['wh_marketing_website_event_tracks_fact']}}

In [17]:
params = {
    'parent_table':parent_table,
    'project_id': project_name,
    'schema_id': schema_name, 
    'table_names': final_list,
    'table_names_unqouted': flat_list,
    'pk_fk_join_key_list': join_key_list
    
}

NameError: name 'parent_table' is not defined

In [33]:
parent_table = (explores.get("parent_table"))


In [34]:
parent_table

In [22]:
explores

{'sales_explore': {'parent_table': ['wh_marketing_website_event_pages_fact'],
  'dimensions': ['wh_marketing_website_users_dim'],
  'facts': ['wh_marketing_website_event_tracks_fact']}}

In [27]:
def deep_get(dictionary, keys, default=None):
    return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)

In [29]:
from functools import reduce

In [32]:
print (deep_get(enviroment_project, "explores.parent_table"))

None


In [94]:
parent_table = []

for key,value in explores.items():

    for key,value in value.items():

        parent_table.append(value[0])

In [95]:
parent_table[0]

'wh_marketing_website_event_tracks_fact'

In [89]:
res = parent_table

In [53]:
print (enviroment_project.get('explores', {}).get('parent_table'))

None


In [51]:
enviroment_project

{'profile': 'ra-development',
 'explores': {'sales_explore': {'parent_table': ['wh_marketing_website_event_pages_fact'],
   'dimensions': ['wh_marketing_website_users_dim'],
   'facts': ['wh_marketing_website_event_tracks_fact']}}}

In [55]:
print(enviroment_project['explores']['parent_table'])

KeyError: 'parent_table'